In [ ]:
#r "nuget: TorchSharp-cpu"
using TorchSharp;
using static TorchSharp.TensorExtensionMethods;
using Microsoft.DotNet.Interactive.Formatting;

var style = TensorStringStyle.Julia;

Formatter.SetPreferredMimeTypesFor(typeof(torch.Tensor), "text/plain");
Formatter.Register<torch.Tensor>((torch.Tensor x) => x.ToString(style, newLine: "\n"));

# Tensors

In TorchSharp, as in all deep learning, the fundamental data type is a 'tensor,' which is simply a generalized matrix. In Linear Algebra, a one-dimensional arrays are called 'vector,' and a two-dimensional array is a 'matrix.' Generalizing on that, a tensor is simply an N-dimensional array. 

Please note that there is an overloaded use of the word 'dimensions' here -- in physics, a vector (one dimension) with three elements is used to represents a point in space, one element for each spatial dimension. When we speak of 'dimension' in these tutorials, it is the number of tensor dimensions that is of interest.

So, let's get started with tensors by creating some.

## Constant-Filled Tensors

The simplest tensor creation primitives just initializes a tensor with either 0 or 1 in all its elements. The arguments passed in is the size of each dimension. Think of the first dimension as the rows of a table, the second as the columns, and then you just have to generalize things in your head after that. In the examples below, we'll mostly be creating 3x4 matrices, for simplicity.

One thing to note is that .NET Interactive will show the object and its fields, etc. when you say just 't' at the end of the notebook cell. What we want for tensors is to show the contents, and there's a special version of ToString() taking a Boolean that shows not just the size and type of the tensor, but also its contents. The special .NET Interactive formatter that is found at the top of the notebook (and should be at the top of all notebooks using TorchSharp) uses TorString(true).

In [ ]:
var t = torch.ones(3,4);
t

If you have more than two dimensions, the special version of ToString(t) will try to format it in a way that makes sense to a human:

In [ ]:
torch.ones(2,4,4)

If you intend to fill the tensor with values from somewhere else, in other words because you pre-allocated it, there's an 'empty' factory that is faster than using anything else. The values are just whatever was found in memory when the tensor was created. Don't mistake that for random values, though -- they won't adhere to any particular distribution. In fact, one of the main uses for empty tensors is to create one, and then fill it with random numbers from some particular distribution.

In [ ]:
torch.empty(4,4)

You can also create a tensor from any value you want:

In [ ]:
torch.full(4,4,3.14f)

Sometimes, you want to display more than one value in a cell. To do that, you need to rely on good-old-fashioned printing:

In [ ]:
Console.WriteLine(torch.zeros(4,4).ToString(style, newLine: "\n"));
Console.Write(torch.ones(4,4).ToString(style, newLine: "\n"));

All that typing will get pretty tedious, so we added an extension method `print()` that is much quicker to type. It really should only be used in notebooks:

In [ ]:
torch.zeros(4,4).print(style);
torch.ones(4,4).print(style);

You may have noticed that each tensor has a 'type = Float32' attribute. This is a peculiarity about the TorchSharp tensor type -- the element type does not show up in the type, Tensor is not Tensor\<T\>. This is so because the underlying C++ / CUDA runtime represents tensors this way, and it makes it easier to port code from Python, too.

You may also have noticed that tensors are created using factories, not constructors. Also, the naming convention doesn't look anything like .NET. We chose to step away from .NET conventions in order to make it easier to port code from Python. We know this will upset some, and please some, but it's the decision we came to after a long time of deliberating.

Anyway, 'Float32' is the default, but you can create tensors of other types, too, including complex tensors:

In [ ]:
torch.zeros(4,15, dtype: torch.int32)

In [ ]:
t = torch.zeros(4,4, dtype: torch.complex64)

To access the contents of a tensor, you treat it as a multi-dimensional array (note that the number of dimensions also isn't part of the type itself). When you do, you'll be surprised to see that the result of the indexing operator is another tensor, one that has no shape -- this is how TorchSharp represents a scalar value. Later in this tutorial, we will see why. For now, just know that you have to extract the value using a function, based on the type you expect to get out.

In PyTorch, there's a method '.item()' used for this purpose. In TorchSharp, it's a templatized method: 

In [ ]:
t = torch.zeros(4,4, dtype: torch.int32);
Console.Write(t[0,0]);
t[0,0].item<int>()

To write to a single element, you have to create a tensor from the value you want to write.

In [ ]:
t[0,0] = torch.tensor(35);
t

## Randomized Tensors

In machine learning, random number generation is very important, and you often end up using the RNG APIs to create tensors. There are a big number of RNGs, most of them for floating point values, but there are some for integers, too.

The usual suspects are present -- normal and uniform distributions, binomial (true/false or 0/1) and uniformly distributed integers. A separate tutorial will disscuss random number generation in more detail.

In [ ]:
// Normal distribution
torch.randn(3,4)

In [ ]:
// Uniform distribution between [0,1[
torch.rand(3,4)

To change the range, just multiply and/or add:

In [ ]:
// Uniform distribution between [100,110[
(torch.rand(3,4) * 10 + 100)

The main factory for integer values is not quite as convenient to use -- in the function signature, there is an integer to pass in for the max value, so the dimension values have to be passed in an array or tuple. In C#, the tuple literal syntax is more concise than array literals:

In [ ]:
torch.randint(10, (3,4))

There's a lot more to know about random number generation -- TorchSharp offers a great many distributions and way to initialize tensor. This space is so rich, it has its own tutorial coming up later.

# Ranges

Another common and convenient tensor factory is `arange()`, which is used to create a 1D tensor with numbers ranging from a min to a max, exclusive of the max. You can provide the step value, or let it be the default, which is 1:

In [ ]:
torch.arange(3,19)

As you can see, `arange` behaves a little differently from other factories -- the default element type is not float32, it's inferred from the arguments. To use floating-point, we an either specify the type, or use floating-point values as arguments:

In [ ]:
torch.arange(3.0f, 14.0f)

The step argument will allow us to get a more fine-grained number series:

In [ ]:
torch.arange(3.0f, 5.0f, step: 0.1f)

## reshape()

There is no way to make `arange` produce anything but a 1D tensor, so a common thing to do is to reshape it as soon as it's created.

In [ ]:
torch.arange(3.0f, 5.0f, step: 0.1f).reshape(4,5)

When printing out floating-point tensors, it is often helpful to provide a formatting string, applied only to FP elements, including the real and imaginary parts of complex numbers. `.ToString(true, "0.00")` would be a lot to type every time, so TorcSharp defines a shorter (and Python-looking) method `str()` that does the trick. `print()` also takes a format string.

In [ ]:
torch.arange(3.0f, 5.0f, step: 0.1f).reshape(4,5).str(fltFormat: "0.00")

`reshape()` is, of course, useful for many other things, too, not just shaping the result of `arange()`. One thing that is very useful to know is that you can pass in '-1' for __one__ of the dimensions, and it has a very special meaning. Let's look at an example:

In [ ]:
t = torch.rand(3,4,4,4);
t.reshape(12, 4, 4).ToString()

In [ ]:
t.reshape(-1,4,4).ToString()

In [ ]:
t.reshape(4,-1,6).ToString()

As you can see, -1 is a wildcard. After the rest of the arguments specify their respective sizes, the -1 dimension is determined from the overall number of elements and the dimensions that have been specified. Obviously, it can only be used to construct a proper tensor if the other dimensions are correct. This, for example, results in an exception:

In [ ]:
t.reshape(4,-1,5).ToString()

# Common Tensor Properties and Operations

There is a large number of operations on tensors. Here are some of the most commonly used ones.

In [ ]:
t = torch.arange(3.0f, 5.0f, step: 0.1f).reshape(2,2,5);

// The overall shape of the tensor:
t.shape

In [ ]:
// The number of dimensions:
t.ndim

In [ ]:
// The total number of elements held in the tensor:
t.numel()

In [ ]:
// Move the tensor to the GPU and back to the CPU:

// t.cuda() // Uncomment if you're using one of the CUDA backends.
t.cpu()

In [ ]:
// Getting the transpose of a matrix:
torch.arange(3.0f, 5.0f, step: 0.1f).reshape(4,5).print();
torch.arange(3.0f, 5.0f, step: 0.1f).reshape(4,5).T.print();

It's important to note that transposing isn't the same as simply reshaping to a different shape:

In [ ]:
torch.arange(3.0f, 5.0f, step: 0.1f).reshape(5,4).print("0.00");
torch.arange(3.0f, 5.0f, step: 0.1f).reshape(4,5).T.print("0.00");

If you need to have some idea of how many tensors you have allocated in you process, there's a static property `TotalCount` that tells you just how many .NET tensors are active. That's not all tensors, just those that have a .NET representation. Temporaries used by the native library and not surfaced to managed code don't count.

The property is useful if you are diagnosing memory issues, for example in a training loop. If the number of tensors keeps growing, somewhere there's a missing Dispose() call.

In [ ]:
torch.Tensor.TotalCount

There's also a `PeakCount` static property that keeps track of the high-water mark:

In [ ]:
torch.Tensor.PeakCount

Create a copy of a tensor by calling `clone()`. The new tensor will have its own backing storage space.

In [ ]:
// Clone a tensor:
var s = t.clone();
s[0,0,1] = torch.tensor(375);
s.print();
t

Create a new managed code reference to the same underlying tensor data by calling `alias()`. This is useful when a function needs to return an input tensor, but the caller has reason to expect a fresh reference. For example, the caller may assume that you can call `Dispose()` on the resulting tensor reference.

In [ ]:
var a = t.alias();
a[0,0,1] = torch.tensor(250);
t

TorchSharp has great support for complex tensors, which are as easy to create as others. Complex numbers are commonly used in signal processing scenarios, such as audio analysis. Complex numbers have two components -- the real and the imaginary parts.

In [ ]:
var ct = torch.rand(3,4,dtype:torch.ScalarType.ComplexFloat32);
ct.str(fltFormat:"0.00")

To get the real and imaginary parts, there are property accessors for that:

In [ ]:
ct.real

In [ ]:
ct.imag

When you create random complex tensors, both the real and imaginary parts are random. However, when you create a tensor with `torch.ones`, only the real part is filled in -- the imaginary part is '0'

In [ ]:
torch.ones(3,4,dtype:torch.ScalarType.ComplexFloat32)

Experienced PyTorch users will notice that TorchSharp doesn't represent complex numbers in the str() output exactly the same: TorchSharp uses 'i' instead of 'j' to indicate the imaginary part, and we don't print the real or imaginary part if it is zero (unless they are both zero, of course).